In [1]:
#Single task network to compare with Vlado
import numpy as np
import pandas as pd
import torch
import scipy as sp

# from bayes_opt import BayesianOptimization
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from math import sqrt
import random
import itertools



In [2]:
#Load some of the data
exp_data = pd.read_csv('../exp.tab', sep='\t', index_col=0)
cnv_data = pd.read_csv('../cnv.tab', sep='\t', index_col=0)
ydat = pd.read_csv('../labels.tab', sep='\t', index_col=0)
train_activity_data = pd.read_csv('../train_activity.tab', sep='\t')
test_activity_data = pd.read_csv('../test_activity.tab', sep ='\t')

#labels
traininglabels = train_activity_data.columns[1:]
testinglabels = test_activity_data.columns[1:]

#concatenate two data frames
frames = [exp_data, cnv_data]

xdatw = pd.concat(frames)
# traininglabels


In [12]:
#Deep Learning Net Class

class EssentialityNet:

    def __init__(self):
        self.inputnum = xdatw.shape[0]
        self.trainscores = []
        self.testscoreslist = []
        self.learning_rate = 0.00009
        self.H = 100
        self.n_iter = 300 #training iterations
        self.minimum = 100000
        self.stopcounter = 3
        self.layernum = 1
        self.layers = []
                
        #model
        self.model = torch.nn.Sequential(
        torch.nn.Linear(self.inputnum, self.H),
        torch.nn.ReLU(),
        torch.nn.Linear(self.H, 1),
        )
        
        #set loss function and optimizer
        self.loss = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
    
    #plot scores
    def plot(self, trainscores, testscores):
        x = np.arange(self.n_iter)
        plt.plot(x, self.trainscores, label='Train')
        plt.title('Training vs Test Accuracy')
        plt.xlabel('NN Training Iterations')
        plt.ylabel('Accuracy')
    
        plt.plot(np.asarray(x), np.asarray(testscores), label='Test') #plot
        plt.legend()
        
    #sets the proper method
    def setModel(self, Layernum, Neuronnum):  
        
        self.layernum = int(round(Layernum))
        self.H = int(round(Neuronnum))
        
        #initial input layer
        self.layers.append(torch.nn.Linear(self.inputnum, self.H))
        
        for n in range(self.layernum):
            if n != 0:
                self.layers.append(torch.nn.Linear(self.H, self.H))
            self.layers.append(torch.nn.ReLU())
            
        self.layers.append(torch.nn.Linear(self.H, 1))
        
        #set the method to whatever layers were chosen
        self.model = torch.nn.Sequential(*self.layers)
    
    def setRegularization(self, L2Reg):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay= L2Reg)

    def fit(self, xtrain, ytrain, xtest, ytest):
      
        #convert to variables
        xtrain_var = Variable(torch.FloatTensor(xtrain))
        xtest_var = Variable(torch.FloatTensor(xtest))
        ytrain_var = Variable(torch.FloatTensor(ytrain))
        ytest_var = Variable(torch.FloatTensor(ytest))
        
        for t in range(self.n_iter):
        
            #calculate loss
            ypred = self.model(xtrain_var)

            diff = self.loss(ypred, ytrain_var)
            self.trainscores.append(diff.data[0])
            
            #test performance
            ypredtest = self.model(xtest_var)
            difftest = self.loss(ypredtest, ytest_var)
            
            #find the best point
            if t > 10 and self.minimum < difftest.data[0]:
                self.stopcounter -= 1

                if self.stopcounter == 0:
                    self.n_iter = t
                    self.trainscores.pop()
                    break
            elif t > 10 and self.stopcounter < 3:
                self.stopcounter += 1
            
            self.minimum = difftest.data[0]
            
            self.testscoreslist.append(difftest.data[0])
            
            #zero gradients
            self.optimizer.zero_grad()
            #backpropagate
            diff.backward() 
            #update weights
            self.optimizer.step() 

    # predict with the test data
    def predict(self, X):
        
        X_var = Variable(torch.FloatTensor(X))
        predicts = self.model(X_var)
        return predicts.data.numpy()
    
#other functions for running the nn

def figureoutnetwork(layernum, neuronnum, l2reg):
    n = EssentialityNet()
    n.setModel(layernum, neuronnum)
    n.setRegularization(l2reg)
            
    n.fit(xtrain_val, ytrain_val, xtest_val, ytest_val)
    predictions = n.predict(xtest)
    
    rmse = calculateRMSE_single(predictions, ytest)
    cor = calculateCor_single(predictions, ytest)
    
    return([rmse, cor])
 
#calculate RMSE function
def calculateRMSE_single(predicts, actuals):
    
    #unpack list
    predicts2 = [val for sublist in predicts for val in sublist]

    return sqrt(((predicts2 - actuals)**2).mean())

#calculate RMSE function for multitask
def calculateRMSE(predicts, actuals):
    mses = []  
    multitaskrmses = []
    preds = predicts.data.numpy()

    for i in range(preds.shape[1]):
        mses.append(((preds[:,i] - actuals[:,i])**2).mean())
        multitaskrmses.append(sqrt(mses[i]))

    print(len(multitaskrmses))       
    return(np.mean(multitaskrmses))

def calculateCor_single(predicts, actuals):
    
    predicts2 = [val for sublist in predicts for val in sublist]

    cor = str(sp.stats.spearmanr(actuals, predicts2))
    cor = float(cor[28:37])

    return(cor)

    
    
def saveRMSE(predicts, actuals):
    mses = []  
    multitaskrmses = []
    preds = predicts.data.numpy()

    for i in range(preds.shape[1]):
        mses.append(((preds[:,i] - actuals[:,i])**2).mean())
        multitaskrmses.append(sqrt(mses[i]))
    
    #open a file for saving rmses
    rmses_file = open('rmses_single_' + str(fileno) + ".tab", 'w')
    
    for item in multitaskrmses:
          rmses_file.write("%s\n" % item)

In [38]:
#run for first 5 genes
mselist = {}

#best tasks that vlado tested with
topTasks = pd.read_csv("../combined_stats.tab", sep='\t')
tasks = topTasks.iloc[:,0].values

#keep the tasks we want
goodydat = ydat.transpose()[tasks]
goodydat = goodydat.transpose()

goodydat


(1138,)


,22RV1_PROSTATE,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,786O_KIDNEY,A1207_CENTRAL_NERVOUS_SYSTEM,A172_CENTRAL_NERVOUS_SYSTEM,A204_SOFT_TISSUE,A2058_SKIN,A549_LUNG,A673_BONE,ACHN_KIDNEY,...,TT_OESOPHAGUS,TYKNU_OVARY,U178_CENTRAL_NERVOUS_SYSTEM,U251MG_CENTRAL_NERVOUS_SYSTEM,U343_CENTRAL_NERVOUS_SYSTEM,U87MG_CENTRAL_NERVOUS_SYSTEM,UOK101_KIDNEY,VCAP_PROSTATE,YKG1_CENTRAL_NERVOUS_SYSTEM,ZR7530_BREAST
Name,,,,,,,,,,,,,,,,,,,,,
AADAT_1_11010,-1.233717,1.630825,-0.609781,0.220788,0.520919,1.625103,0.744893,0.270954,0.195283,-0.272367,...,0.624118,-0.073269,-0.682420,-1.471325,0.474586,-0.102329,0.020837,0.699519,-0.047015,0.501435
AASDHPPT_1_10110,-0.835507,-1.841625,-1.041909,0.811463,-1.411331,-0.816945,0.704509,-0.099023,-0.475277,-0.056040,...,0.590706,1.004681,-0.192678,-1.153011,-0.715164,-0.447077,0.877866,-1.726210,-0.523734,1.534973
ABCB1_1_11100,-0.077366,-0.192850,-0.534267,-1.739908,-0.266028,-1.287434,0.636443,-0.222219,-0.414539,-1.004412,...,-0.440657,-0.490540,-1.532204,-0.036978,-2.164719,-0.754494,-0.053760,0.292978,0.800539,1.019134
ABCB5_1_01010,-0.350602,-0.001090,0.885487,-1.330497,-1.576496,0.007458,-0.320978,0.268817,-1.284329,-0.794162,...,-0.319307,-0.576655,-1.315208,0.174838,-0.285739,-2.067271,-1.392602,0.854448,-0.866509,1.195604
ABCB7_1_10111,-0.540359,0.677912,-0.196409,2.752970,-0.362563,1.428641,-0.349743,-0.920291,-1.576565,1.981206,...,-0.085135,1.494078,0.830243,0.628817,0.806812,1.016900,-0.112952,0.229912,0.286890,-1.220823
ABCC3_1_0011,-1.493482,-0.556698,-1.184588,1.005566,0.938299,0.995693,-1.058201,1.758713,-1.218867,-0.771106,...,-0.080481,2.289804,-0.403979,-1.160949,2.247304,0.888143,-0.199145,0.315864,2.000316,-0.906171
ABLIM3_1_01111,0.656319,1.238755,-0.583299,-0.422763,0.403560,0.388922,-0.698032,-0.872150,2.355338,0.009699,...,1.700019,-1.262018,0.083041,-1.060057,0.182273,-1.022443,-0.948695,0.989110,-0.792063,0.562011
ACADL_1_10011,0.811633,-0.603420,-0.771375,1.260552,-2.119105,-1.282944,-0.234332,-0.845653,-1.007366,-0.727710,...,-1.411534,0.281601,0.624200,-0.080234,1.767618,1.192108,0.689886,-0.561078,0.697050,1.550448
ACCS_1_01010,0.845706,0.336642,-1.489349,-1.187804,-0.336668,-0.980081,0.177349,-0.903707,0.426053,-0.140524,...,-0.461968,-0.020557,0.493230,-1.216529,-0.377021,-0.795705,-0.266741,-0.510991,-0.327985,1.618282


In [46]:

#open file to save the rmses and correlations
rmses_file = open('rmses_singles.tab', 'w')  

for i in range(1138):
    
    #initialize
    n = EssentialityNet()
          
    #get name
    name = goodydat.index[i]
    
    #get training and testing labels that vlado used
    traininglabelsdat = pd.read_csv('../train_test_splits/train_test_splits/' + name + '_train_idx', header = None)
    testinglabelsdat = pd.read_csv('../train_test_splits/train_test_splits/' + name + '_test_idx', header = None)
    
    #convert them to lists
    traininglabels = traininglabelsdat.iloc[:,0].values
    testinglabels = testinglabelsdat.iloc[:,0].values

    #index the data 
    xtraindat = xdatw[traininglabels].transpose()
    xtestdat = xdatw[testinglabels].transpose()
    ytraindat = goodydat[traininglabels].transpose()
    ytestdat = goodydat[testinglabels].transpose()
    
    #normalize inputs
    xtrain = preprocessing.scale(xtraindat)
    xtest = preprocessing.scale(xtestdat)

    #get column (task)
    ytrain = np.array(ytraindat.iloc[:,i])  
    ytest = np.array(ytestdat.iloc[:,i]) 
    
    
    #split for validation
    xtrain_val, xtest_val, ytrain_val, ytest_val = train_test_split(xtrain, ytrain, test_size=0.2, random_state=434)
    
    print(i)

    #fit and predict
    rmse, cor = figureoutnetwork(3, 100, 0.012)

    rmses_file.write(name + '\t' + str(rmse) + '\t' + str(cor) + '\n')

#close the file
rmses_file.close() 
   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [44]:
rmses_file.close() 


In [66]:
#run different architectures for random tasks (type of grid search)
for i in range(10):
    
    i = random.randint(0,1137)
    
    #initialize
    n = EssentialityNet()
          
    #best tasks
    topTasks = pd.read_csv("../combined_stats.tab", sep='\t')
    tasks = topTasks.iloc[:,0].values
    
    goodydat = ydat.transpose()[tasks]
    goodydat = goodydat.transpose()
    
    #get name
    name = goodydat.index[i]
    
    #get training and testing labels that vlado used
    traininglabelsdat = pd.read_csv('../train_test_splits/train_test_splits/' + name + '_train_idx', header = None)
    testinglabelsdat = pd.read_csv('../train_test_splits/train_test_splits/' + name + '_test_idx', header = None)
    
    #convert them to lists
    traininglabels = traininglabelsdat.iloc[:,0].values
    testinglabels = testinglabelsdat.iloc[:,0].values

    #index the data 
    xtraindat = xdatw[traininglabels].transpose()
    xtestdat = xdatw[testinglabels].transpose()
    ytraindat = goodydat[traininglabels].transpose()
    ytestdat = goodydat[testinglabels].transpose()
    
    #normalize inputs
    xtrain = preprocessing.scale(xtraindat)
    xtest = preprocessing.scale(xtestdat)

    #get column (task)
    ytrain = np.array(ytraindat.iloc[:,i])  
    ytest = np.array(ytestdat.iloc[:,i]) 
    
#     print(xtrain)
#     print(ytrain)
#     print(xtrain_val)
    
    #split for validation
    xtrain_val, xtest_val, ytrain_val, ytest_val = train_test_split(xtrain, ytrain, test_size=0.2, random_state=434)
    
#     print(i)
    print(name)

    #fit and predict
#     print("RMSE: ")
#     print(figureoutnetwork(1, 100, 0.012))
#     print(figureoutnetwork(1, 356, 0.012))
#     print(figureoutnetwork(2, 100, 0.012))
#     print(figureoutnetwork(2, 356, 0.012))
    r1,c1 = figureoutnetwork(3, 100, 0.012)
#     print(figureoutnetwork(3, 356, 0.012))
    r2,c2 = figureoutnetwork(4, 100, 0.012)
#     print(figureoutnetwork(4, 356, 0.012))
    print(r1 - r2)
    #3 356
#     torch.save(n, "/home/user/Documents/stuart_research/data/data/exp_cnv_activity/single_models/" + name + ".txt")

WNK1_1_1101
0.046456113673984056
XCL2_1_001100010
-0.005767808501719651
PRRG4_1_010001
-0.014177578990969497
EIF4G1_1_0101
-0.061999411677636784
OR51V1_1_11000
0.025279173199407867
CD47_1_00011
0.04411266838812433
MXD4_1_00101
0.07114159918875473
CDK6_1_100111111100
0.03245727181920233
IL26_1_01101
0.04482142779439535
GRHL2_1_10001
0.011060282600278692
